In [80]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from pandas_gbq import to_gbq

In [81]:
# Carregue suas credenciais
credencial = service_account.Credentials.from_service_account_file(
    "/Users/juniorsilva/Downloads/python-curso-459117-fc7b944c47c6.json", scopes=['https://www.googleapis.com/auth/bigquery'])

In [82]:
# Nome do projeto e dataset
projeto = 'python-curso-459117'
dataset = 'Ecommerce'

In [83]:
# Mapear arquivos com nome da tabela
arquivos = {
    'produto': '/Users/juniorsilva/develop/data/estudos-engenharia-dados/engenharia-dados-com-python-e-banco-de-dados/Google BigQuery/Arquivos/Produto.xlsx',
    'ordens': '/Users/juniorsilva/develop/data/estudos-engenharia-dados/engenharia-dados-com-python-e-banco-de-dados/Google BigQuery/Arquivos/Ordens.xlsx',
    'items': '/Users/juniorsilva/develop/data/estudos-engenharia-dados/engenharia-dados-com-python-e-banco-de-dados/Google BigQuery/Arquivos/items.xlsx',
    'categoria': '/Users/juniorsilva/develop/data/estudos-engenharia-dados/engenharia-dados-com-python-e-banco-de-dados/Google BigQuery/Arquivos/Categoria.xlsx',
    'clientes': '/Users/juniorsilva/develop/data/estudos-engenharia-dados/engenharia-dados-com-python-e-banco-de-dados/Google BigQuery/Arquivos/Clientes.csv'
}

In [84]:
for tabela, caminho in arquivos.items():
    if caminho.endswith('.csv'):
        df = pd.read_csv(caminho)
    elif caminho.endswith('.xlsx'):
        df = pd.read_excel(caminho)
    else:
        print(f'Formato não suportado da tabela {tabela}')
        continue

    #print(df.head)
    print(f"Tabela: {tabela} - Colunas:",df.columns.tolist())


Tabela: produto - Colunas: ['ID', 'Name', 'Price', 'Id_Category']
Tabela: ordens - Colunas: ['id', 'created_at', 'customer_id', 'status']
Tabela: items - Colunas: ['id', 'order_id', 'product_id', 'quantity', 'total_price']
Tabela: categoria - Colunas: ['id', 'name']
Tabela: clientes - Colunas: ['id', 'created_at', 'first_name', 'last_name', 'email', 'cell_phone', 'country', 'state', 'street', 'number', 'additionals']


In [85]:
# Função para padronizar os nomes das colunas para minúsculas e com underscores
def padronizar_colunas(df):
    df.columns = [col.lower().replace(' ', '_') for col in df.columns] # Converter todas as colunas para minúsculas e substituir espaços por underscores
    return df

In [86]:
# Função para transformação de Produto
def transformar_produto(df):
    df['price'] = df['price'].astype(float) # Garantir que 'price' seja float
    df['id_category'] = df['id_category'].astype(int) # Garantir que 'id_category' seja inteiro
    df['name'] = df['name'].str.strip().str.title()  # Tornar o 'name' mais uniforme
    return df

# Função para transformação de Ordens
def transformar_ordens(df):
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['customer_id'] = df['customer_id'].astype(int)  # Garantir que 'customer_id' seja int
    return df

# Função para transformação de Items
def transformar_items(df):
    df['order_id'] = df['order_id'].astype(int)  # Garantir que 'order_id' seja int
    df['product_id'] = df['product_id'].astype(int)  # Garantir que 'product_id' seja int
    df['quantity'] = df['quantity'].astype(int)  # Garantir que 'quantity' seja int
    df['total_price'] = df['total_price'].astype(float)  # Garantir que 'total_price' seja float
    return df

# Função para transformação de Categoria
def transformar_categoria(df):
    df['name'] = df['name'].str.strip() # Remove espaços em branco do início e fim de cada string
    return df

# Função para transformação de Clientes
def transformar_clientes(df):
    df['first_name'] = df['first_name'].str.strip().str.title() # Garantir que 'first_name' não tenha espaços e a primeira letra seja maiuscula
    df['last_name'] = df['last_name'].str.strip().str.title() # Garantir que 'last_name' não tenha espaços e a primeira letra seja maiuscula
    df['email'] = df['email'].fillna('Não informado').astype(str).str.lower()  # Padronizar 'email' para minúsculo
    df['email'] = df['email'].replace('não informado', 'Não informado')
    df['number'] = df['number'].astype(float) # Garantir que 'number' seja float

    # Tratar valores ausentes (NaN) em algumas colunas opcionais
    colunas_opcionais = ['cell_phone', 'country', 'state', 'street', 'number', 'additionals']
    for coluna in colunas_opcionais:
        if coluna in df.columns:
            df[coluna] = df[coluna].fillna('Não informado')  # Garante que toda coluna que for NaN seja substituido
    return df

In [ ]:
# Função para carregar e enviar com chamada de transformação
def carregar_e_enviar(tabela, caminho, projeto, dataset, credencial, if_exists='replace'):
    print(f'Carregando: {tabela}')
    
    if caminho.endswith('.csv'):
        df = pd.read_csv(caminho, sep=',')  # Define o separador aqui
    elif caminho.endswith('.xlsx'):
        df = pd.read_excel(caminho)
    else:
        raise ValueError(f'Formato não suportado: {caminho}')
    
    df = padronizar_colunas(df)

    # Aplica transformação específica por nome da tabela
    if tabela == 'produto':
        df = transformar_produto(df)
    elif tabela == 'ordens':
        df = transformar_ordens(df)
    elif tabela == 'items':
        df = transformar_items(df)
    elif tabela == 'categoria':
        df = transformar_categoria(df)
    elif tabela == 'clientes':
        df = transformar_clientes(df)

    destino = f'{projeto}.{dataset}.{tabela}'
    print(f'Enviando para BigQuery: {destino}')
    
    df.to_gbq(destination_table=destino,
              project_id=projeto,
              if_exists=if_exists,
              credentials=credencial)

In [88]:
for tabela, caminho in arquivos.items():
    carregar_e_enviar(tabela, caminho, projeto, dataset, credencial)

Carregando: produto
Enviando para BigQuery: python-curso-459117.Ecommerce.produto


/var/folders/3r/bw0_ywj11pb_yrqsny5pgw480000gn/T/ipykernel_9387/1586387839.py:29: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=destino,


Carregando: ordens
Enviando para BigQuery: python-curso-459117.Ecommerce.ordens


/var/folders/3r/bw0_ywj11pb_yrqsny5pgw480000gn/T/ipykernel_9387/1586387839.py:29: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=destino,


Carregando: items
Enviando para BigQuery: python-curso-459117.Ecommerce.items


/var/folders/3r/bw0_ywj11pb_yrqsny5pgw480000gn/T/ipykernel_9387/1586387839.py:29: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=destino,


Carregando: categoria
Enviando para BigQuery: python-curso-459117.Ecommerce.categoria


/var/folders/3r/bw0_ywj11pb_yrqsny5pgw480000gn/T/ipykernel_9387/1586387839.py:29: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=destino,


Carregando: clientes
Enviando para BigQuery: python-curso-459117.Ecommerce.clientes


/var/folders/3r/bw0_ywj11pb_yrqsny5pgw480000gn/T/ipykernel_9387/1586387839.py:29: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=destino,
